In [ ]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import torch.optim as optim
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

In [ ]:
def prepare_gru_data(self, data, feature_columns, target_column, sequence_length=50):
    """
    Prepare the data for the gru model, by making sequence of features and the targets
    """
    X, y = [], []
    
    # Normalize the feature columns
    scaler = MinMaxScaler()
    data[feature_columns] = scaler.fit_transform(data[feature_columns])
    
    for i in range(len(data) - sequence_length):
        X.append(data[feature_columns].iloc[i:i + sequence_length].values)
        y.append(data[target_column].iloc[i + sequence_length])
    
    return np.array(X), np.array(y)

In [ ]:
class GRUModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super().__init__()
        self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out, _ = self.gru(x)
        out = self.fc(out[:, -1, :])  # use last hidden
        return out

## Functions Above!

In [ ]:
# Set up the data
feature_columns = ['SMA_20', 'SMA_50', 'RSI', 'MACD', 'Volume']  
target_column = 'Action' 
sequence_length = 50

X, y = prepare_gru_data(data, feature_columns, target_column, sequence_length)

In [ ]:
# Model parameters
input_size = X.shape[2]  # Number of features
hidden_size = 64
num_layers = 2
output_size = 1  

model = GRUModel(input_size, hidden_size, num_layers, output_size)

# Define loss and optimizer
criterion = nn.BCEWithLogitsLoss()  # BCE for classification
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# Split data and then convert them to tensors
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

# Training loop
epochs = 20
batch_size = 32

for epoch in range(epochs):
    model.train()
    for i in range(0, len(X_train_tensor), batch_size):
        X_batch = X_train_tensor[i:i + batch_size]
        y_batch = y_train_tensor[i:i + batch_size]
        
        optimizer.zero_grad()
        outputs = model(X_batch).squeeze()
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()
    
    print(f"Epoch {epoch + 1}/{epochs}, Loss: {loss.item()}")

In [ ]:
# Quick evaluation to see
model.eval()
with torch.no_grad():
    y_pred = model(X_test_tensor).squeeze()
    y_pred_class = (torch.sigmoid(y_pred) > 0.5).float()

# Calculate accuracy
accuracy = (y_pred_class == y_test_tensor).float().mean()
print(f"Test Accuracy: {accuracy.item()}")

# Visualize predictions
plt.figure(figsize=(12, 6))
plt.plot(y_test, label="Actual")
plt.plot(torch.sigmoid(y_pred).numpy(), label="Predicted", alpha=0.7)
plt.legend()
plt.show()